In [4]:
#read extraction results
import pandas as pd
from pathlib import Path
import glob
pathfixo = Path().absolute()
pathextração = ('Data extraction\\')
pastaextração = Path.joinpath(pathfixo, pathextração)
pathinput = ('Data input\\')
pastainput = Path.joinpath(pathfixo, pathinput)


files_extraction = glob.glob(f"{pastaextração}\\*.json")
df = []
for f in files_extraction:
    json = pd.read_json(f)
    df.append(json)
df_extraction = pd.concat(df, ignore_index=True)

higher_similarity = df_extraction.groupby(['productId'])['similarity_ratio'].max().reset_index()

dfclean = pd.merge(df_extraction,higher_similarity,how='inner',  on= ['productId','similarity_ratio'])

dfclean = dfclean.drop(dfclean.loc[(dfclean['similarity_ratio'] <= 0.01) | (dfclean['rating'] < 2)].index)

#dfclean.head()

In [5]:
# read input rows

input = pd.read_json(f'{pastainput}\\result.json')
#input = pd.read_json(r'C:\Users\fabio\OneDrive\Python\Playwright_Scraping\Data input\result.json')
df = pd.json_normalize(input['data']['productSearch']['products'])
item_list = []
for row in df.iterrows():
    item_detail = {
        'productId' : row[1]['productId'],
        'productName' : row[1]['productName'],
        'itemId' : row[1]['items'][0]['itemId'],
        'ean' : row[1]['items'][0]['ean'],
        'sellername' : row[1]['items'][0]['sellers'][0]['sellerName'],
        'ListPrice' : row[1]['items'][0]['sellers'][0]['commertialOffer']['ListPrice']
    }   
    item_list.append(item_detail)
df = pd.DataFrame(item_list)
df = df.astype({'productId':'int64'})
#df.head()


In [6]:
df2 = pd.merge(df, dfclean, on='productId', validate='one_to_many')
df2[['productId', 'productName','ListPrice','title','current_price', 'url', 'similarity_ratio']].to_excel('resultados.xlsx')
#df2.head()